In [1]:
!pip install sparqlwrapper

  Using cached SPARQLWrapper-2.0.0-py3-none-any.whl (28 kB)
  Using cached rdflib-6.1.1-py3-none-any.whl (482 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)


In this project, we will be combinining quarterly sales data of Apple iPhones with respective Apple stock prices. This should give us a broad overview over the development of the company. I have added comments to the Python code, below to make it easy to understand. Please bear in mind, that the comments were tranlsated from German to English, as this project was submitted at an Austrian university (100% of points achieved). This is also personally my first introduction to Sparql.

In [19]:
import requests
from datetime import datetime
# https://data.world/rflprr/iphone-sales
# iPhone Verkäufe pro Quartal in Million 
r = requests.get('https://query.data.world/s/h3netlxe3y2msx7i6io6g7m7pixdvl')
f = open('iphone-sales.csv', 'wb')
f.write(r.content)
f.close()

In [20]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [21]:
%sql sqlite:///iphones.db

In [22]:
%%sql
drop table if exists sales;

 * sqlite:///iphones.db
Done.


[]

In [23]:
import sqlite3
import pandas
conn = sqlite3.connect('iphones.db')

In [24]:
pandas.read_csv('iphone-sales.csv').to_sql('sales', conn, index = False)

In [25]:
%%sql
select * from sales limit 5;

 * sqlite:///iphones.db
Done.


Category,iPhone,YOY growth
Q2/07,0.141,None
Q3/07,0.489,None
Q4/07,1.036,None
Q1/08,0.817,None
Q2/08,0.483,242.55


In [26]:
%%sql
drop table if exists stock;

 * sqlite:///iphones.db
Done.


[]

- Category is the respective quarter (1-4) with the respective Year next to it (e.g. 08 = 2008)
- "iPhone" is a string respenting the number of sold iPhones in Millions iPhone (e.g., 0.141 = 141,000 sold iPhones in the quarter)
- YoY Growth can be ignored for our analysis (Year over Year Growth)

In [27]:
# https://www.macrotrends.net/stocks/charts/AAPL/apple/stock-price-history
# apple aktienkurs von 1980 bis 2022, täglicher wert
r = requests.get('https://www.macrotrends.net/assets/php/stock_data_download.php?s=629725c636b34&t=AAPL')
f = open('apple-stock.csv', 'w')
x = r.text.find('date,open')
length = len(r.text)
f.write(r.text[x:(length-1)])
f.close()
# 

In [28]:
pandas.read_csv('apple-stock.csv').to_sql('stock', conn, index = False)

- Date is the tag of the price. 
- Open is the opening price of Apple's stock (on the New York Stock Exchange), i.e. the first trade of the day. 
- High ist the highest trading valye during the day. 
- Low ist the lowest trading value during the day.
- Closing ist the last trade of the day. 

In [29]:
%%sql
select * from stock limit 5;

 * sqlite:///iphones.db
Done.


date,open,high,low,close,volume
1980-12-12,0.1004,0.1008,0.1004,0.1004,469033600
1980-12-15,0.0956,0.0956,0.0952,0.0952,175884800
1980-12-16,0.0886,0.0886,0.0882,0.0882,105728000
1980-12-17,0.0904,0.0908,0.0904,0.0904,86441600
1980-12-18,0.093,0.0934,0.093,0.093,73449600


In [30]:
%%sql
drop table if exists dates;

 * sqlite:///iphones.db
Done.


[]

In [31]:
import math
quartal_to_first_of_month = []
z = 0



while(z <= 36):
    # the first quarter in the dataset is 02, we start with (z + 1)
    # (z+1)%4 gives us a number between 0-3, quarters range from 1-4; therefore +1 
    q = (z+1)%4 + 1
    # we need for for each 4 consecutive quarters, always the same year, starting with 2007 
    year = math.trunc((z+1)/4)+7
    yearS = str(year)
    # year is always in double-digits in the dataset, if we have the year 7,8 or 9, we need an extra 0
    if(year < 10):
        yearS = '0' + str(yearS)
    # we calculate the quarter after the current quarter
    qAfter = (z+1)%4 + 1
    # calculation for the jahr of z+1
    yearAfter = math.trunc((z+2)/4)+7
    # calculation of the first month, after the current quarter 
    monthAfter = (qAfter * 3) % 12 + 1
    monthAfterS = str(monthAfter)
    yearAfterS = '20' + str(yearAfter)
    # if the month is not double-digits (one-digit), convert to doubel digits 
    if(monthAfter < 10):
        monthAfterS = '0' + str(monthAfter)
    # if the year is 7,8 or 9, attach an extra 0
    if(yearAfter < 10):
        yearAfterS = '200' + str(yearAfter)
    # increment z 
    z += 1
    # append the entry in the list 
    quartal_to_first_of_month.append(['Q' + str(q) + '/' + yearS, str(yearAfterS) + '-' + str(monthAfterS) + '-01'])

# convert the list to a dataframe, so that, to_sql can be used to create a table with this list 
pandas.DataFrame(quartal_to_first_of_month, columns=['quartal', 'dayAfterQuartal']).to_sql('dates', conn, index = False)


Here a two-dimensional array is created where the quarters are in the first column the quarters are entered and in the second column, the first day after the quarter is entered

This table is required to create an association between the two tables

In [32]:
%%sql
select s.date, s.open, s.close, sa.iPhone * (1000000) from dates d join stock s on d.dayAfterQuartal = s.date join sales sa on d.quartal = sa.category limit 5

 * sqlite:///iphones.db
Done.


date,open,close,sa.iPhone * (1000000)
2007-10-01,4.7213,4.7735,489000.0
2008-04-01,4.467,4.5656,817000.0
2008-07-01,5.0089,5.3335,483000.0
2008-10-01,3.4172,3.3317,4406000.0
2009-04-01,3.1782,3.3186,2427000.0


In [33]:
%%sql
drop table if exists iphoneReleases;

 * sqlite:///iphones.db
Done.


[]

In [34]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV


In [35]:
endpoint_url = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(endpoint_url)

In [36]:
query = """SELECT DISTINCT ?item ?label ?releaseDate WHERE {
  ?item p:P279 ?statement0.
  ?statement0 (ps:P279/(wdt:P279*)) wd:Q2766.
  ?item wdt:P571 ?releaseDate.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en".
          ?item rdfs:label         ?label.
      }
}
order by ?releaseDate"""

sparql.setQuery(query)

In [37]:
sparql.setReturnFormat(JSON)

In [38]:
iphones = sparql.query().convert()

In [39]:
iphoneReleases = []
for res in iphones["results"]["bindings"]:
    row = []
    row.append(res["label"]["value"])
    row.append(res["releaseDate"]["value"].split("T")[0])
    iphoneReleases.append(row)

iphoneReleases    
    
pandas.DataFrame(iphoneReleases, columns=['name', 'releaseDate']).to_sql('iphoneReleases', conn, index = False)

In [40]:
%%sql
select s.date, s.open, s.close, sa.iPhone * (1000000) "IPhone Sales", (select name from iphoneReleases where releaseDate < s.date order by releaseDate desc) "Newest Iphone"
  from dates d
  join stock s on d.dayAfterQuartal = s.date 
  join sales sa on d.quartal = sa.category

 * sqlite:///iphones.db
Done.


date,open,close,IPhone Sales,Newest Iphone
2007-10-01,4.7213,4.7735,489000.0,iPhone
2008-04-01,4.467,4.5656,817000.0,iPhone
2008-07-01,5.0089,5.3335,483000.0,iPhone 3G
2008-10-01,3.4172,3.3317,4406000.0,iPhone 3G
2009-04-01,3.1782,3.3186,2427000.0,iPhone 3G
2009-07-01,4.3815,4.361,3060000.0,iPhone 3GS
2009-10-01,5.6593,5.5222,4606000.0,iPhone 3GS
2010-04-01,7.2512,7.2048,5445000.0,iPhone 3GS
2010-07-01,7.7645,7.5868,5334000.0,iPhone 4
2010-10-01,8.6637,8.6261,8822000.0,iPhone 4


In this table, I have now combined the share price (closing and open of the trading day) on the date of the respective quarter with the sales of iPhones in that quarter. 
Furthermore, the latest iPhone model is displayed in the last column. This indicates the general trend that Apple's stock price has risen as iPhone sales have increased.